In [92]:
import numpy as np
import matplotlib.pyplot as plt

In [93]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [94]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std 

In [95]:
X_st = X.copy()
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [96]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i]) 
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

In [97]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [98]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [99]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [100]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [101]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

Вручную

In [102]:
W, error = eval_model(X_st, y, iterations=100, eta=5)

0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.760958797591889
10 [-0.01029206 -2.82809091 -0.88447098  3.73398716] 0.3226261607632078
20 [ 0.18002003 -2.97134197 -1.34779553  4.84469069] 0.27715351986139203
30 [ 0.31866054 -3.19436082 -1.77479772  5.62385571] 0.2593025845433026
40 [ 0.4446571  -3.41604738 -2.12394358  6.28495733] 0.24655287466702386
50 [ 0.55910135 -3.62730214 -2.42417791  6.87809585] 0.23640505725582975
60 [ 0.66466222 -3.82833485 -2.69302119  7.42403877] 0.22786606495433842
70 [ 0.76314938 -4.02006992 -2.93980244  7.93421816] 0.22044019616890415
80 [ 0.85575995 -4.20344727 -3.16984754  8.41566398] 0.21384584337320178
90 [ 0.94333118 -4.37929796 -3.38645036  8.87303097] 0.20790689534007237


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [103]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [104]:
calc_pred_proba(W, X_st)

array([3.58043233e-01, 1.13341370e-01, 9.99605449e-01, 4.39572170e-04,
       7.46305694e-01, 2.33000018e-01, 9.99914868e-01, 8.92559045e-02,
       4.46315218e-01, 9.97517198e-01])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [105]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [106]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

Accuracy

In [107]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [108]:
print(f'True \t {y}')
print(f'Predicted  {pred}')

True 	 [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
Predicted  [0 0 1 0 1 0 1 0 0 1]


In [109]:
accuracy(y, pred)

0.9

Confusion matrix

In [110]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)):
        if y[i] == y_pred[i] == 1:  # TP
                cm[0][0] += 1
        elif y[i] == y_pred[i] == 0:   # TN
                cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1:  # FN
                cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0:  # FP
                cm[0][1] += 1
    return cm                

In [111]:
cm = confusion_matrix(y, pred)
cm

array([[4., 0.],
       [1., 5.]])

Precision

In [112]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FN = cm[1][0]
    FP = cm[0][1]
    precision = TP / (TP + FP)
    return precision

In [113]:
precision(y, pred)

1.0

Recall

In [114]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FN = cm[1][0]
    FP = cm[0][1]
    recall = TP / (TP + FN)
    return recall

In [115]:
recall(y, pred)

0.8

F-score

In [116]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [117]:
f_score(y, pred)

0.888888888888889

6. Могла ли модель переобучиться? Почему?

Нет, так как метрики не очень высокие.